In [ ]:
!pip install openai

In [ ]:
import openai
import getpass
from openai import OpenAI
import re

In [ ]:
# Prompt the user for the API key
api_key = getpass.getpass("Enter your OpenAI API key: ")

openai.api_key = api_key

Enter your OpenAI API key: ··········


In [ ]:
client = OpenAI(
    api_key=api_key,
)


In [ ]:
dietary_restrictions = {
    1: "None",
    2: "Keto",
    3: "Vegan",
    4: "Vegetarian",
    5: "Gluten-Free",
    6: "Lactose-Intolerant",
    7: "Paleo",
    8: "Halal",
    9: "Other"
}

goals = {
    1: "Weight Loss",
    2: "Weight Gain",
    3: "Muscle Gain",
    4: "Maintenance",
    5: "Other"
}

def _convert_weight_to_kg(response):
        lbs_kg_pattern = re.compile(r"(\d+)\s*(kg|lb(s)?)?", re.IGNORECASE)
        match = lbs_kg_pattern.search(response)
        if match:
            value, unit = int(match.group(1)), match.group(2)
            if not unit or "kg" in unit.lower():
                return f"{value}kg"
            elif "lb" in unit.lower():
                return f"{value / 2.20462:.0f}kg"

        return response

def _convert_height_to_cm(response):
        cm_inches_pattern = re.compile(r"(\d+)\s*(cm|inch(es)?)?", re.IGNORECASE)
        match = cm_inches_pattern.search(response)
        if match:
            value, unit = int(match.group(1)), match.group(2)
            if not unit or "cm" in unit.lower():
                return f"{value}cm"
            elif "inch" in unit.lower():
                return f"{value * 2.54:.0f}cm"

        return response

def generate(prompt):
    try:
        openai.api_key = api_key
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "You are an intelligent health profile assistant. Your primary task is to gather information about the user (including weight, height, goal)"
                },
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return str(e)


def generate_update(prompt):
    try:
        openai.api_key = api_key
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI designed to assist with updating a user's health profile based on their input regarding changes. Identify and categorize changes mentioned by the user into fields such as weight, height, dietary restrictions, gender, and goal.",
                },
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return str(e)

# Specializing Agents
def generate_calorie(prompt):
    try:
        openai.api_key = api_key
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful calorie assistent to help users calculate their daily intake based on their profile. Please give an estimation if there is not enough information."
                },
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return str(e)

# Calculate daily maintanennce calories

def generate_recipe(prompt):
    try:
        openai.api_key = api_key
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": "You are an intelligent meal planner. Your primary goal is to recommend recipes (including ingredients, steps, and nutrition information) based on the user's profile."
                },
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
        )
        return completion.choices[0].message.content
    except Exception as e:
        return str(e)

class HealthProfileAssistant:
    def __init__(self):
        self.user_profile = {
            "height": None,
            "weight": None,
            "gender": None,
            "dietary_restriction": None,
            "goal": None
        }
        self.questions = {
            "height": "Could you tell me your height? (e.g., 170cm or 5'7\")",
            "weight": "What's your weight? (e.g., 70kg or 155lbs)",
            "gender": "What's your gender?",
            "dietary_restriction": "Do you have any dietary restrictions? (e.g., vegan, keto)",
            "goal": "What's your goal? (e.g., weight loss, maintenance, muscle gain)"
        }

    def validate_and_process_response(self, topic, response):
        if topic == "dietary_restriction":
            prompt = f"""
            Given the user's response '{response}' regarding their dietary restrictions, categorize their diet using the embedded dictionary:
            {dietary_restrictions}
            Please only return the number(s) associated with the closest dietary restriction category. If other, please specify.
            """
            response = generate(prompt)
            numbers = re.findall(r'\d+', response)
            restriction_numbers = [int(num) for num in numbers]
            if len(restriction_numbers) == 0:
              return True, response
            else:
              restriction_names = [dietary_restrictions[number] for number in restriction_numbers if number in dietary_restrictions]
              unique_restriction_names = list(set(restriction_names))
              return True, ", ".join(unique_restriction_names)
        else:
            if topic in ["height", "weight"]:
                prompt = f"From this response '{response}', can we tell what the user's {topic} is and the unit? Please answer Yes or No, and if Yes, provide the user's {topic}."
            else:
                prompt = f"From this response '{response}', can we tell what the user's {topic} is? Please answer Yes or No, and if Yes, provide the user's {topic}."

            ai_response = generate(prompt)

            if "no" in ai_response.lower().split() and topic != "dietary_restriction":
                prompt = (
                    f"The user has provided a response '{response}' to the topic '{topic}'. "
                    "It appears that the response may not have the detail or specificity required. "
                    "Please help the user with inputting a valid response and give any specific guidance on their invalid input"
                )
                return False, response
            else:
                processed_response = self.process_ai_response(topic, ai_response)
                return True, processed_response

    def process_ai_response(self, topic, response):
        if topic in ["height", "weight"]:
            prompt = f"Convert the user's {topic} '{response}' into standard units (write cm for height, kg for weight)."
            calculations = generate(prompt)
            prompt = f"Given the user's {topic} is {calculations}, please only return their final {topic} with standard units(write cm for height, kg for weight)."
        elif topic == "gender":
            prompt = f"Based on this response '{response}', how can we best classify the user's gender? Consider traditional categories like 'male' or 'female', or say other."
        elif topic == "goal":
            prompt = f"Given this response '{response}', what is the user's fitness or health goal? Classify it into one of the following categories {goals}. Simply return the number associated with the closest goal. Or summarize their goal if 'Other'"
            response = generate(prompt)
            numbers = re.findall(r'\d+', response)
            goal_numbers = [int(num) for num in numbers]
            unique_goal_numbers = list(set(goal_numbers))
            goal_names = [goals[number] for number in unique_goal_numbers if number in goals]
            return ", ".join(goal_names)
        else:
            return self.finalize_response(topic, response)
        ai_interpreted_response = generate(prompt)
        return self.finalize_response(topic, ai_interpreted_response)

    def finalize_response(self, topic, response):
        if topic == "height":
            return _convert_height_to_cm(response)
        elif topic == "weight":
            return _convert_weight_to_kg(response)
        elif topic == "gender":
            return self._interpret_gender(response)
        else:
          return response

    def _interpret_gender(self, response):
        return next((value for word in response.lower().split() for key, value in {
            "female": "Female",
            "male": "Male",
        }.items() if key in word), "Prefer not to specify")


    def is_complete(self):
        return all(value is not None for value in self.user_profile.values())

def specify_updates(updates, field_name):
        prompt = f"The user wants to make the following changes to their health profile '{updates}', write a conversational question asking them to give specifics about their changes to their {field_name}. Only reply with the question and only ask about their {field_name}."
        response = generate_update(prompt)
        return response


def summarize_updates(specified_update, user_input, field_name):
        prompt = f"The user responded {user_input} to the question {specified_update} about their new {field_name}, please summarize what changes there were to their {field_name}"
        return generate_update(prompt)

class UpdateAssistant:
    def __init__(self, user_profile):
            self.user_profile = user_profile
            self.field_names = list(self.user_profile.keys())

    def check_for_update_intent(self, user_input):
          prompt = (
              f"Based on the following user response: '{user_input}' about if they want to update their health profile, say yes if the user is expressing a desire to update their health profile. "
          )
          response = generate_update(prompt)
          return "yes" in response.lower()

    def identify_fields_to_update(self, updates):
        """Generate a prompt to identify which profile fields are mentioned in the updates."""
        fields_prompt = '\n'.join([f"{idx + 1}. {name.capitalize()}" for idx, name in enumerate(self.field_names)])
        prompt = f"""Given the user profile fields:
                {fields_prompt}
                And the user's updates: "{updates}", identify which profile fields are mentioned in the updates. Only list the fields by their numbers. Return 0 if no change needed."""
        response = generate_update(prompt)
        return response

    def extract_fields_to_update(self, response):
    # Use regex to find all numbers in the response
        return [int(num) for num in re.findall(r'\d+', response)]

    def process_updates(self, updates, field_name):
        if field_name in ["height", "weight"]:
            prompt = (
                f"Given the user's update that they '{updates}' which implies a change in their {field_name}, "
                f"and their current {field_name} is {self.user_profile[field_name]}, calculate the new {field_name}. "
                f"Convert all measurements to the unit of the current {field_name} before calculating. "
            )
            calculations = generate_update(prompt)
            prompt = f"Given the following calculations for {field_name}: {calculations}, what is the user's final {field_name}? Return only return the final {field_name} rounded to the nearest whole number and with the unit"
            response = generate_update(prompt)
            response = _convert_weight_to_kg(response) if field_name == "weight" else _convert_height_to_cm(response)
        elif field_name == "dietary_restriction":
            prompt = (
                f"Analyze the user's response '{updates}' and update the dietary restriction accordingly. "
                f"The current dietary restriction is {self.user_profile[field_name]}. Possible categories include: "
                "None, Keto, Vegan, Vegetarian, Gluten-Free, Lactose-Intolerant, Paleo, Halal. Or specify if other"
                "Please only return the updated category."
            )
            response = generate_update(prompt)
            response = _convert_weight_to_kg(response)
        elif field_name == "goal":
            prompt = (
                f"Analyze the user's response '{updates}' and update the goal accordingly. "
                f"The current goal is {self.user_profile[field_name]}. Possible goals include: "
                "Weight maintenance, Muscle Gain, Weight loss. "
                f"Please only return the updated goal. Return the original {field_name} if no updates can be made."
            )
            response = generate_update(prompt)
        else:
            prompt = f"Analyze the user's response '{updates}. update their {field_name}. The current {field_name} is {self.user_profile[field_name]}. Please only return the updated value. Return the original {field_name} if no updates can be made."
            response = generate_update(prompt)
        return response


    def calculate_calories(self):
        height = self.user_profile["height"]
        weight = self.user_profile["weight"]
        gender = self.user_profile["gender"]
        goal = self.user_profile["goal"]
        dietary_restriction = self.user_profile["dietary_restriction"]

        prompt = (
            f"Calculate the recommended daily goal calorie intake for the user with the following profile: "
            f"Height: {height}, Weight: {weight}, Gender: {gender}, "
            f"Dietary Restriction: {dietary_restriction}, Goal: {goal}. "
        )
        response = generate_calorie(prompt)
        prompt = (
            f"Based on the following analysis{response}, list the user's recommended daily intake as a whole number, do not include any units or extra text."
        )
        return generate_calorie(prompt)

class RecipeAssistent:
    def __init__(self, user_profile, recipe_description):
        self.client = client
        self.messages = [
            {
                "role": "system",
                "content": f"You are a helpful meal assistent and you will consider the users' profile {user_profile} when providing recommendations about the current recipe {recipe_description}. You will help with user queries related to meal planning and provide helpful responses."
            }
        ]

    def generate(self, user_prompt):
        self.messages.append({"role": "user", "content": user_prompt})

        completion = self.client.chat.completions.create(
            model="gpt-4",
            messages=self.messages
        )

        response_content = completion.choices[0].message.content

        self.messages.append({"role": "assistant", "content": response_content})

        return response_content


In [ ]:
print("Welcome to the Meal Planner Assistant. I'll ask you a few questions to understand your preferences and needs.")
assistant = HealthProfileAssistant()
while not assistant.is_complete():
    for key, question in assistant.questions.items():
        valid_response = False
        while not valid_response:
            if assistant.user_profile[key] is None:
                print(question)
                response = input().strip()
                valid_response, processed_response = assistant.validate_and_process_response(key, response)
                if valid_response:
                    assistant.user_profile[key] = processed_response
                else:
                    print(f"Invalid response for {key}. Please try again.")
print("Thank you for providing the information. Here's your profile:")
for key, value in assistant.user_profile.items():
    print(f"{key.replace('_', ' ').title()}: {value}")
user_profile = assistant.user_profile

Welcome to the Meal Planner Assistant. I'll ask you a few questions to understand your preferences and needs.
Could you tell me your height? (e.g., 170cm or 5'7")
5'6''
What's your weight? (e.g., 70kg or 155lbs)
120 pounds
What's your gender?
female
Do you have any dietary restrictions? (e.g., vegan, keto)
I am gluten-free
What's your goal? (e.g., weight loss, maintenance, muscle gain)
I want to gain weight
Thank you for providing the information. Here's your profile:
Height: 167cm
Weight: 54kg
Gender: Female
Dietary Restriction: Gluten-Free
Goal: Weight Gain


In [ ]:

print("Let me know if there are any changes to your health profile!")
update_agent = UpdateAssistant(user_profile)
updates = input().strip()
if update_agent.check_for_update_intent(updates):
    print("What are the changes to your health profile?")
    updates = input().strip()
    fields_to_update = update_agent.extract_fields_to_update(update_agent.identify_fields_to_update(updates))
    for field_index in fields_to_update:
        if field_index == 0:
            continue
        field_name = update_agent.field_names[field_index - 1]
        prompt = f"From the user  's response about their change in health profile '{updates}', do we have specific information to update their {field_name}? Simply say yes if we do, otherwise just say No"
        response = generate_update(prompt)
        if "no" in response.lower():
            user_update = specify_updates(updates, field_name)
            print(user_update)
            user_response = input().strip()
            update_agent.user_profile[field_name] = update_agent.process_updates(summarize_updates(user_update, user_response, field_name), field_name)
        else:
            update_agent.user_profile[field_name] = update_agent.process_updates(updates)
    print("Thank you for providing the information. Here's your new profile:")
    for key, value in update_agent.user_profile.items():
        print(f"{key.replace('_', ' ').title()}: {value}")
print("Calculating Calories")
calories = update_agent.calculate_calories()
print(f"Your recommended daily calories is {calories}")
updated_profile = update_agent.user_profile


Let me know if there are any changes to your health profile!
yes
What are the changes to your health profile?
weight
Could you please specify the updates for your weight?
I gained 10 pounds
Thank you for providing the information. Here's your new profile:
Height: 160cm
Weight: 55kg
Gender: Female
Dietary Restriction: None
Goal: Weight Gain
Calculating Calories
Your recommended daily calories is 2598


In [ ]:
print("Let’s find your perfect recipe! Please tell me about what you are looking for in a recipe such as any preferences in taste, cook time, budget, or health considerations. Include any other relevant details. This helps me pick the best recipes for you!")
preferences = input().strip()
prompt = f"Given the user's preferences described as: '{preferences}', summarize these preferences into a concise statement suitable for NLP processing."
user_preference = generate_recipe(prompt)
prompt = f"Given the user profile {user_profile}, and their preference for meals {user_preference}, generate a recipe for them that is around {calories} calories"
recipe_description = generate_recipe(prompt)
print(recipe_description)
print("\n")
print("Any questions about the recipe?")
print("Type 'exit' or 'quit' to end the conversation.\n")

while True:
    recipe_agent = RecipeAssistent(updated_profile, recipe_description)
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Assistant: Goodbye!")
        break
    response = recipe_agent.generate(user_input)
    print("Assistant:", response)

Let’s find your perfect recipe! Please tell me about what you are looking for in a recipe such as any preferences in taste, cook time, budget, or health considerations. Include any other relevant details. This helps me pick the best recipes for you!
I'm looking for a quick meal with avocadoes
Recipe: High-Calorie Avocado Chicken Salad

Servings: 2

Ingredients:
1. 2 large ripe avocados, peeled and diced
2. 2 Chicken breasts, grilled and diced
3. 2 Tablespoons of olive oil
4. 1 cup of mayonnaise 
5. 1 cup of corn kernels
6. 1 cup of sliced almonds
7. 1 Tablespoon of honey
8. Salt and pepper to taste
9. 2 cups of cooked quinoa
10. 2 tablespoons of lemon juice

Instructions:
1. In a large bowl, add the minced chicken breasts, diced avocados, corn kernels, and cooked quinoa.
2. In another bowl, mix the olive oil, mayonnaise, honey, and lemon juice. Stir until well combined to make the dressing.
3. Pour the dressing over the chicken and avocado mixture. Toss until all the ingredients are we